In [0]:
from collections import Counter, defaultdict
import numpy as np

# Корутины

С помощью `yield` можно легко писать бесконечные генераторы.

Например, бесконечный поток чисел Фибоначчи:

In [0]:
def fibonacci_generator():
    a, b = 1, 1
    while True:
        yield a;
        a, b = b, a + b
        
gen = fibonacci_generator()
# print(next(gen))
# print(next(gen))
# print(next(gen))

А если хочется получить сразу несколько первых элементов из генератора?

**Задание** Напишите функцию-генератор, которая будет выдавать элементы до тех пор, пока не наберется нужное количество (`n`).

In [0]:
def take(generator, n):
    for i in range(n):
      yield next(generator)
        
list(take(fibonacci_generator(), 5))

[1, 1, 2, 3, 5]

**Задание** Напишите функцию, которая будет выдавать только четные числа Фибоначчи.

In [0]:
def even_num_generator(num_generator):
    for a in num_generator:
      if a % 2 == 0:
        yield a

list(take(even_num_generator(fibonacci_generator()), 5))

[2, 8, 34, 144, 610]

Другая доступная операция - `send`:  
![send](https://image.ibb.co/dbKyLS/2018_03_29_13_58_49.png=x300)

In [0]:
def receiver():
    while True:
        item = yield
        print('Got', item)

recv = receiver()
next(recv)
recv.send('Hello')
recv.send('World')

Got Hello
Got World


## N-граммная языковая модель

In [0]:
!wget -qq -O perashki.txt https://share.abbyy.com/index.php/s/Y86O2aRLOcdnNWv/download
  
!head perashki.txt

Давайте посчитаем частотности слов в файле:

In [0]:
word_counts = Counter()
with open('perashki.txt', encoding='utf-8') as f:
    for line in f:
        for word in line.strip().split():
            if len(word) != 0:
                word_counts[word] += 1
                
print(word_counts.most_common(100))

[('и', 24199), ('в', 17829), ('не', 10508), ('на', 10145), ('я', 9481), ('а', 8950), ('с', 5802), ('что', 4487), ('как', 4430), ('он', 3343), ('но', 3295), ('у', 2899), ('ты', 2797), ('мне', 2794), ('олег', 2752), ('по', 2707), ('за', 2659), ('из', 2481), ('к', 2382), ('то', 2317), ('вот', 2143), ('когда', 2125), ('все', 1920), ('так', 1639), ('меня', 1627), ('мы', 1600), ('от', 1492), ('его', 1427), ('это', 1343), ('нет', 1302), ('там', 1296), ('всё', 1216), ('под', 1204), ('сказал', 1187), ('же', 1105), ('для', 1063), ('чтоб', 1056), ('был', 1043), ('потом', 1016), ('о', 1011), ('их', 986), ('кто', 980), ('где', 955), ('есть', 952), ('вдруг', 943), ('она', 916), ('теперь', 910), ('оксана', 901), ('вы', 888), ('про', 847), ('ну', 842), ('до', 838), ('без', 825), ('уже', 817), ('только', 807), ('мой', 764), ('бы', 748), ('нас', 676), ('сегодня', 652), ('тебя', 639), ('ни', 633), ('они', 610), ('день', 609), ('да', 593), ('во', 586), ('тут', 584), ('нам', 581), ('если', 575), ('два', 56

Почему бы не разнести чтение файла и получение отдельных токенов и их обработку?

Тогда при чтении легко можно будет добавлять некоторую предобработку - например, токенизацию не по пробелам, а с помощью nltk, не изменяя работу обработчика.

**Задание** Напишите `parser` - функцию, которая будет выдавать поток токенов из файла.

In [0]:
def parser(path):
    with open(path, encoding='utf-8') as f:
      for line in f:
          for word in line.strip().split():
              yield word

pars = parser("perashki.txt")

print(next(pars))
print(next(pars))

старик
вытягивает


Раз у нас есть такая удобная функция, почему бы не использовать её для чего-то интересного.

Напишем N-граммную языковую модель.

Языковая модель - это штука, которая умеет оценивать вероятности $\mathbf{P}(w_1, \ldots, w_n) = \prod_k \mathbf{P}(w_k|w_{k-1}, \ldots, w_{1})$.

N-граммная языковая модель приближает эту вероятность, используя предположение, что вероятность токена зависит только от недавней истории: $\mathbf{P}(w_k|w_1, \ldots, w_{k-1}) = \mathbf{P}(w_k|w_{k-1}, \ldots, w_{k-N + 1})$.

Для начала нужно собрать статистику. Для простоты будем работать с триграммной моделью, а значит - нужно собрать информацию:
- о триграммных частотностях $(w_{i-2}, w_{i-1}) \to C(w_i)$ - то есть о числе раз, когда слово $w_i$ шло за парой $w_{i-2}, w_{i-1}$
- о биграммных частотностях $(w_{i-1}) \to C(w_i)$
- об униграммных частотностях $() \to C(w_i)$

**Задание** Напишите функцию, которая будет из потока токенов формировать и выдавать наружу пары (ngram, next_word).

In [0]:
def compose_ngram(tokens_stream):
  #    <yields trigrams, bigrams and unigrams composed from tokens stream>
  first = next(tokens_stream)
  second = next(tokens_stream)
  for third in tokens_stream:
    yield ((), first)
    yield ((first, ), second)
    yield ((first, second), third)
    first = second
    second = third
 
  

In [0]:
ngrams = compose_ngram(pars)
print(next(ngrams))
print(next(ngrams))
print(next(ngrams))
print(next(ngrams))
print(next(ngrams))
print(next(ngrams))
print(next(ngrams))
print(next(ngrams))
print(next(ngrams))

((), 'и')
(('и',), 'заперла')
(('и', 'заперла'), 'окно')
((), 'заперла')
(('заперла',), 'окно')
(('заперла', 'окно'), 'не')
((), 'окно')
(('окно',), 'не')
(('окно', 'не'), 'дав')


Соберем статистику:

In [0]:
def collect_stat(path):
    ngrams_counter = defaultdict(Counter)
    for ngram in compose_ngram(parser(path)):
        ngrams_counter[ngram[0]][ngram[1]] += 1
    
    return ngrams_counter

ngrams_counter = collect_stat('perashki.txt')

In [0]:
ngrams_counter

defaultdict(collections.Counter,
            {(): Counter({'старик': 41,
                      'вытягивает': 2,
                      'сети': 38,
                      'они': 610,
                      'пусты': 10,
                      'и': 24198,
                      'лишь': 515,
                      'в': 17829,
                      'конце': 74,
                      'записка': 14,
                      'рыба': 51,
                      'недоступна': 2,
                      'или': 510,
                      'вне': 8,
                      'действия': 7,
                      'олег': 2752,
                      'адепт': 7,
                      'шизофрении': 1,
                      'шагает': 21,
                      'бодро': 6,
                      'из': 2481,
                      'окна': 107,
                      'жызнь': 96,
                      'летит': 135,
                      'перед': 295,
                      'глазами': 120,
                      'да': 593,
        

Теперь генерировать будем так: пусть есть некоторый уже сгенерированный набор слов (возможно, пустой).

Тогда проверяем, есть ли статистика для последних двух слов - если есть, генерируем с помощью неё новое. Нет - тогда смотрим статистику для только одного слова. И так далее.

In [0]:
def sample_token(ngrams_counter, ngram):
    probs = np.array(list(ngrams_counter[ngram].values()))
    probs = probs / np.sum(probs)
    return np.random.choice(list(ngrams_counter[ngram]), p=probs)  
  
def generate_token(ngrams_counter):
    #<generates next token using >
    first = sample_token(ngrams_counter, ())
    if (first, ) in ngrams_counter:
      second = sample_token(ngrams_counter, (first,))
    else:
      second = sample_token(ngrams_counter, ())
    while True:
      if (first, second) in ngrams_counter:
        third = sample_token(ngrams_counter, (first, second))
      elif second in ngrams_counter:
        third = sample_token(ngrams_counter, second)
      else:
        third = sample_token(ngrams_counter, ())
      yield third
      first = second
      second = third

In [0]:
list(take(generate_token(ngrams_counter), 30))

['вы',
 'на',
 'ты',
 'на',
 'глянцевых',
 'журналах',
 'то',
 'на',
 'коралл',
 'то',
 'на',
 'мальков',
 'сегодня',
 'умерла',
 'собака',
 'а',
 'ты',
 'внимательно',
 'прильнул',
 'ко',
 'рту',
 'великого',
 'вождя',
 'давайте',
 'в',
 'городе',
 'осталось',
 'не',
 'двадцать',
 'шесть']